# SBASInvert.py 

This is the simplest time-series inversion method that is included with **GIAnT**.

In [1]:
cd /home/jovyan/work/notebooks/TimeSeries/GRFN/GIAnT

/home/jovyan/work/notebooks/TimeSeries/GRFN/GIAnT


## 1. Preparing for SBAS

Before running an **SBAS** inversion, we need to setup some related parameters. This is done via the **prepsbasxml.py** script.

Let's take a look at this script that we generated in the PrepGIAnT notebook.

In [2]:
!cat prepsbasxml.py


#!/usr/bin/env python

import tsinsar as ts
import argparse
import numpy as np

if __name__ == '__main__':
    g = ts.TSXML('params')
    g. prepare_sbas_xml(nvalid = 11, 
                        netramp=False,   ##No deramping requested
                        atmos='',        ##No troposphere correction requested
                        demerr = False,  ##No dem error correction requested
                        uwcheck=False,   ##For future, not implemented yet
                        regu=True,       ##For Mints. Not relevant for tutorial
                        filt = 0.05)     ##Filter length in years
    g.writexml('sbas.xml')


### Understanding the parameters

Here is the complete list of parameters available for controlling the SBAS inversion.

<table>
    <thead>
        <tr>
            <td>Parameter</td><td>Type</td><td>Description</td><td>Default</td><td>Comment</td>
        </tr>
    </thead>
    <tr>
        <td>netramp</td><td>bool</td><td>Network deramping</td><td>False</td><td>Used to account for orbit errors. Modeled as a ramp </td>
    </tr>
    <tr>
        <td>atmos</td><td>str</td><td>Weather model for PyAPS</td><td>None</td><td>One of ECMWF, ERA, NARR, MERRA, TROPO</td>
    </tr>
    <tr>
        <td>demerr</td><td>bool</td><td>Estimate DEM error</td><td>False</td><td>Recommended for radar geometry stacks where data has not been filtered heavily. For tight baselines like S1 - very little impact</td>
    </tr>
    <tr>
        <td>nvalid</td><td>int</td><td>Min ifgs where a pixel is coherent</td><td>0</td><td>N-SBAS setting. Not recommended for well connected networks like S1</td>
    </tr>
    <tr>
        <td>regu</td><td>bool</td><td>Use regularization for inversion</td><td>False</td><td>Used with splines</td>
    <tr>
    <tr>
        <td>masterdate</td><td>str</td><td>Reference date for inversion</td><td>None</td><td>If not provided, earliest date will be used</td>
    </tr>
    <tr>
        <td colspan="5"><u>GPS ramp related parameters</u></td>
    </tr>
    <tr>
        <td>gpsramp</td><td>bool</td><td>GPS-based deramping</td><td>False</td><td>Use GPS time-series / velocity to constrain ramps. Not tested in a while. </td>
    </tr>
    <tr><td>stnlist</td></tr>
    <tr><td>stntype</td>
    </tr>
    <tr><td>gpspath</td></tr>
    <tr><td>gpstype</td></tr>
    <tr><td>gpsvert</td></tr>
    <tr><td>gpspreproc</td><td colspan="4">Not tested extensively lately</tr>
    <tr><td>gpsmodel</td></tr>
    <tr><td>gpspad</td></tr>
    <tr><td>gpsmin</td></tr>
    <tr>
        <td colspan="5"><u>Troposphere estimation from data</u></td>
    </tr>
    <tr><td>tropomin</td><tr>
    <tr><td>tropomax</td><td colspan="4">Not tested extensively lately</td></tr>
    <tr><td>tropolooks</td></tr>
</table>

In [3]:
#Run the script
!python prepsbasxml.py

logger - INFO - GIANT Toolbox - v 1.0
logger - INFO - ---------------------
/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
addsubelement - INFO - Creating Field: nvalid , Value=11, Type=INT
addsubelement - INFO - Creating Field: uwcheck , Value=False, Type=BOOL
addsubelement - INFO - Creating Field: netramp , Value=False, Type=BOOL
addsubel

In [4]:
#Run the python script
!python prepsbasxml.py

logger - INFO - GIANT Toolbox - v 1.0
logger - INFO - ---------------------
/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
addsubelement - INFO - Creating Field: nvalid , Value=11, Type=INT
addsubelement - INFO - Creating Field: uwcheck , Value=False, Type=BOOL
addsubelement - INFO - Creating Field: netramp , Value=False, Type=BOOL
addsubel

In [5]:
#Look at the output
!cat sbas.xml

<params>
  <proc>
    <nvalid>
      <value>11</value>
      <type>INT</type>
      <help>Minimum number of coherent IFGs for a single pixel. If zero, pixel should be coherent in all IFGs.</help>
    </nvalid>
    <uwcheck>
      <value>False</value>
      <type>BOOL</type>
    </uwcheck>
    <netramp>
      <value>False</value>
      <type>BOOL</type>
      <help>Network deramp. Remove ramps from IFGs in a network sense.</help>
    </netramp>
    <gpsramp>
      <value>False</value>
      <type>BOOL</type>
      <help>GPS deramping. Use GPS network information to correct ramps.</help>
    </gpsramp>
    <stnlist>
      <value></value>
      <type>STR</type>
      <help>Station list for position of GPS stations.</help>
    </stnlist>
    <stntype>
      <value></value>
      <type>STR</type>
      <help>Type of station list. False for Sopac, True for (Name,Lat,Lon), velocity for (Name,lat,lon,ve,vn,vu)</help>
    </stntype>
    <gpspath>
      <value></value>
      <type>STR</type>
   

You will see that the **sbas.xml** has been populated with more fields than what you had set. This is a deliberate choice to document all the parameters that were used in the processing - default values in the software are subject to change.

## 2. Understanding SBAS

Let's work through a very simple synthetic problem to understand the SBAS algorithm implemented in GIAnT.

Consider four SAR acquisitions- call them **A, B, C** and **D**. 

Let's assume that we have the following interferograms are used - **AB, AC, BC, BD** and **CD**.

The corresponding connectivity matrix (**Jmat** in GIAnT) can then be written as 

$$J = \left \lgroup \matrix{ 1 & -1 & 0 & 0\cr 1 & 0 & -1 & 0\cr 0 & 1 & -1 & 0\cr 0 & 1 & 0 & -1\cr 0 & 0 & 1 & -1 } \right \rgroup$$

Let's choose scene **A** as the reference and then our inversion problem reduces to 

$$\left \lgroup \matrix{\phi_{AB} \cr \phi_{AC} \cr \phi_{BC} \cr \phi_{BD} \cr \phi_{CD} } \right \rgroup = \left \lgroup \matrix{ -1 & 0 & 0\cr 0 & -1 & 0\cr 1 & -1 & 0\cr 1 & 0 & -1\cr 0 & 1 & -1 } \right \rgroup \cdot \left \lgroup \matrix{\phi_{B} \cr \phi_{C} \cr \phi_{D} } \right \rgroup$$

This is a simple least-squares problem that we can solve with functions already available in numpy / scipy. The key is that the matrix be full rank - else the matrix is ill-constrained.

## 3. Run SBASInvert.py

In [6]:
!SBASInvert.py

logger - INFO - GIANT Toolbox - v 1.0
logger - INFO - ---------------------
/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from ._conv import register_converters as _register_converters
/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/__init__.py:45: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import h5a, h5d, h5ds, h5f, h5fd, h5g, h5r, h5s, h5t, h5p, h5z
/opt/conda/envs/python2/lib/python2.7/site-packages/h5py/_hl/group.py:22: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .. import h5g, h5i, h5o, h5r, h5t, h5l, h5p
<module> - INFO - Number of interferograms = 19
<module> - INFO - Number of unique SAR scenes = 10
<module> - INFO - Number of connected components in network: 1
Timefn - INFO - Adding 10 linear pieces (SBAS)
<module> -

## 4. Looking at the outputs

In [7]:
##List contents 
!ls -ltr 

total 11428
-rw-r--r-- 1 jovyan users      532 Aug 16 03:32 ifg.list
-rw-r--r-- 1 jovyan users      110 Aug 16 03:32 example.rsc
-rw-r--r-- 1 jovyan users      248 Aug 16 03:33 userfn.py
-rw-r--r-- 1 jovyan users 11664000 Aug 16 03:34 watermask.flt
-rw-r--r-- 1 jovyan users      767 Aug 16 03:34 prepdataxml.py
-rw-r--r-- 1 jovyan users      643 Aug 16 03:34 prepsbasxml.py
-rw-r--r-- 1 jovyan users     5554 Aug 16 03:40 data.xml
-rw-r--r-- 1 jovyan users      451 Aug 16 03:40 userfn.pyc
drwxr-sr-x 3 jovyan users       28 Aug 16 03:40 Figs
-rw-r--r-- 1 jovyan users     3313 Aug 16 03:58 sbas.xml
drwxr-sr-x 2 jovyan users       58 Aug 16 04:00 Stack


In [8]:
##List contents of Stack
!ls Stack/*

Stack/LS-PARAMS.h5  Stack/RAW-STACK.h5


### Exploring contents of the HDF5 file from command line

In [10]:
##Using HDF5 tools
!h5ls Stack/LS-PARAMS.h5

cmask                    Dataset {1620, 1800}
dates                    Dataset {10}
mName                    Dataset {10}
masterind                Dataset {SCALAR}
parms                    Dataset {1620, 1800, 9}
rawts                    Dataset {10, 1620, 1800}
recons                   Dataset {10, 1620, 1800}
regF                     Dataset {10}
tims                     Dataset {10}


In [11]:
##More verbose
!h5ls -v Stack/LS-PARAMS.h5

Opened "Stack/LS-PARAMS.h5" with sec2 driver.
cmask                    Dataset {1620/1620, 1800/1800}
    Attribute: help scalar
        Type:      variable-length null-terminated ASCII string
        Data:  "Common pixel mask."
    Location:  1:7512
    Links:     1
    Storage:   11664000 logical bytes, 11664000 allocated bytes, 100.00% utilization
    Type:      native float
dates                    Dataset {10/10}
    Attribute: help scalar
        Type:      variable-length null-terminated ASCII string
        Data:  "Ordinal values of SAR acquisition dates."
    Location:  1:7240
    Links:     1
    Storage:   80 logical bytes, 80 allocated bytes, 100.00% utilization
    Type:      native double
mName                    Dataset {10/10}
    Attribute: help scalar
        Type:      variable-length null-terminated ASCII string
        Data:  "Unique names for model parameters."
    Location:  1:6424
    Links:     1
    Storage:   80 logical bytes, 80 allocated bytes, 100.00% util

In [12]:
##Using gdal
!gdalinfo Stack/LS-PARAMS.h5

Driver: HDF5/Hierarchical Data Format Release 5
Files: Stack/LS-PARAMS.h5
Size is 512, 512
Coordinate System is `'
Metadata:
  cmask_help=Common pixel mask.
  dates_help=Ordinal values of SAR acquisition dates.
  help=Regular SBAS Inversion results. Raw + filtered series.
  parms_help=Stores all the relevant parameters from the SBAS inversion.
  rawts_help=Raw unfiltered time-series
  recons_help=Reconstructed time-series
  regF_help=regF family indicator for model parameters.
  tims_help=SAR acquisition time in years.
Subdatasets:
  SUBDATASET_1_NAME=HDF5:"Stack/LS-PARAMS.h5"://cmask
  SUBDATASET_1_DESC=[1620x1800] //cmask (32-bit floating-point)
  SUBDATASET_2_NAME=HDF5:"Stack/LS-PARAMS.h5"://parms
  SUBDATASET_2_DESC=[1620x1800x9] //parms (32-bit floating-point)
  SUBDATASET_3_NAME=HDF5:"Stack/LS-PARAMS.h5"://rawts
  SUBDATASET_3_DESC=[10x1620x1800] //rawts (32-bit floating-point)
  SUBDATASET_4_NAME=HDF5:"Stack/LS-PARAMS.h5"://recons
  SUBDATASET_4_DESC=[10x1620x1800] //recons (32-

### Lets move on to the plotts_notebook to look at the estimated time-series